Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [80]:
from itertools import product
from random import random, randint, shuffle, seed, choice
import numpy as np
from scipy import sparse
from copy import copy
from functools import reduce
from collections import namedtuple
import math

In [6]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points * 2654435761 + num_sets + density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets - 1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [35]:
NUM_POINTS = [100, 1_000, 5_000]
NUM_SETS = NUM_POINTS
DENSITY = [.3, .7]


Element at row=42 and column=42: True


## Hill Climbing

In [79]:
def fitness(state, sets):
    selected_sets = [i for i, t in enumerate(state) if t]
    items_covered = np.any(sets[selected_sets], axis=0)
    return np.all(items_covered), -sum(state) if np.all(items_covered) else 0


def tweak(state, problem_size):
    new_state = copy(state)
    index = randint(0, problem_size - 1)
    new_state[index] = not new_state[index]
    return new_state


def hill_climbing(current_state, sets, problem_size):
    counter = 0
    best_state = current_state
    best_fitness = fitness(current_state, sets)
    while True:
        counter += 1
        new_state = tweak(current_state, problem_size)
        new_fitness = fitness(new_state, sets)

        if new_fitness >= best_fitness:
            best_state = new_state
            best_fitness = new_fitness

        if new_fitness >= fitness(current_state, sets):
            current_state = new_state

        if new_fitness[0]:
            return current_state, fitness(current_state, sets), counter

        current_state = best_state


total_calls = 0
NUMBER_OF_TESTS = 5

for size in NUM_POINTS:
    for density in DENSITY:
        total_calls = 0
        for _ in range(NUMBER_OF_TESTS):
            x = make_set_covering_problem(size, size, density)
            sets = x.toarray()
            State = namedtuple("State", ["taken", "not_taken"])
            current_state = [choice([False]) for _ in range(size)]
            _, _, calls = hill_climbing(current_state, sets, size)
            total_calls += calls

        average_calls = total_calls / NUMBER_OF_TESTS
        print(f"Number of points: {size}; Density: {density}; Avg Calls: {calls}")

Number of points: 100; Density: 0.3; Avg Calls: 12
Number of points: 100; Density: 0.7; Avg Calls: 4
Number of points: 1000; Density: 0.3; Avg Calls: 21
Number of points: 1000; Density: 0.7; Avg Calls: 6
Number of points: 5000; Density: 0.3; Avg Calls: 24
Number of points: 5000; Density: 0.7; Avg Calls: 7


# Simulated Annealing

In [81]:
def simulated_annealing(initial_state, sets, problem_size):
    initial_temp = 90
    final_temp = 0.1
    alpha = 0.01

    current_temp = initial_temp
    current_state = initial_state
    best_state = current_state

    while current_temp > final_temp:
        new_state = tweak(current_state, problem_size)

        current_fitness, _ = fitness(current_state, sets)
        new_fitness, _ = fitness(new_state, sets)

        if new_fitness >= current_fitness:
            current_state = new_state
            best_state = new_state
        else:
            probability = math.exp((new_fitness - current_fitness) / current_temp)
            if random.random() < probability:
                current_state = new_state

        current_temp -= alpha

    return best_state

def fitness(state, sets):
    selected_sets = [i for i, t in enumerate(state) if t]
    items_covered = np.any(sets[selected_sets], axis=0)
    return np.all(items_covered), -sum(state) if np.all(items_covered) else 0

def tweak(state, problem_size):
    new_state = state.copy()
    index = randint(0, problem_size - 1)
    new_state[index] = not new_state[index]
    return new_state

total_calls = 0
NUMBER_OF_TESTS = 5

for size in NUM_POINTS:
    for density in DENSITY:
        total_calls = 0
        for _ in range(NUMBER_OF_TESTS):
            x = make_set_covering_problem(size, size, density)
            sets = x.toarray()
            State = namedtuple("State", ["taken", "not_taken"])
            current_state = [choice([False]) for _ in range(size)]
            _, _, calls = simulated_annealing(current_state, sets, size)
            total_calls += calls

        average_calls = total_calls / NUMBER_OF_TESTS
        print(f"Number of points: {size}; Density: {density}; Avg Calls: {calls}")

AttributeError: 'builtin_function_or_method' object has no attribute 'randint'